# C7：高级文本生成技术和工具

* LangChain
* DSPy
* Haystack


In [49]:
#%pip install langchain langchain-community llama-cpp-python langgraph duckduckgo_search
#%pip install 
#%pip install llama-cpp-python
%pip install duckduckgo_search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 49.9 kB/s eta 0:00:00a 0:00:06
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 52.1 kB/s eta 0:00:0000:0200:07
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [duckduckgo_search]
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Donwload model
#!curl -LO 'https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf'

## 链

### 单链架构

提示词模板：`langchain.PromptTemplate`

单链结构一般是一个提示词组件+模型

### 多提示词链式架构

前一个模型的输出，作为后一个的提示词的一部分

In [3]:
from langchain import LlamaCpp

llm = LlamaCpp(
    model_path='Phi-3-mini-4k-instruct-fp16.gguf',
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False,
)

llm.invoke('Hi')

llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h80           (n

' there! I need help crafting a CMake script. It\'s for configuring the installation details of my software project, and it should do several things:\n\n1. Set an install prefix if it hasn\'t been defined by the user, defaulting to "/usr/local". Make sure to trim any trailing slashes from this path.\n2. Define a configuration name based on the build type variable provided, stripping non-alphanumeric characters at the start, or leave it empty if none is given. Also, display this configuration name for clarity.\n3. Allow setting an install component only if one has been specified by the user and inform about which component will be installed with a message.\n4. Include support to optionally grant execute permissions on shared libraries upon installation, which should default to "OFF."\n5. Decide whether the installation is happening due to cross-compiling efforts, with "FALSE" as the standard case unless specified otherwise by an external condition.\n\nI\'d appreciate it if you can draft

In [5]:
from langchain_openai import ChatOpenAI
from deepseek import base_url, api_key


deepseek_model = ChatOpenAI(
    model='deepseek-chat',
    base_url=base_url,
    api_key=api_key,
)

#deepseek_model.invoke('你是一个物理学家。请用严谨的方式介绍量子力学。')

In [6]:
from langchain_core.prompts import PromptTemplate


template = '''<s><|user|>
{input_prompt}<|end|>
<|assistant|>'''

prompt = PromptTemplate(
    template=template,
    input_variables=['input_prompt']
)

basic_chain = prompt | llm

basic_chain.invoke({'input_prompt': 'Hello, please tell me a joke about cat.'})

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


" Why don't cats play poker in the jungle? Too many cheetahs! But seriously, why did the cat sit on the computer? Because it wanted to keep an eye on the mouse! (Just kidding!) Cats are just too good at being mysterious for that.\n\n(Note: The joke plays around with puns and light-hearted humor related to cats without offending anyone.)"

In [7]:
basic_chain.invoke({'input_prompt': 'What is 1 + 1?'})

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


' 1 + 1 equals 2. This is a basic arithmetic addition problem, where you combine one unit with another to get two units in total.'

In [37]:
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from typing_extensions import TypedDict


class StoryTitle(TypedDict):
    title: str
    summary: str


def process_title_output(output, config, *args, **kwargs):
    print(f'output: {output}')
    print(f'config: {config}')
    print(f'args: {args}')
    print(f'kwargs: {kwargs}')
    return {
        'title': output
    }


title_prompt = PromptTemplate(
    template='''<s><|user|>
    Create a title for a story about {summary}. Only return the title.<|end|>
    <|assistant|>''',
    input_variables=['summary']
)
title =  title_prompt | {'summary': RunnablePassthrough(), 'title': llm}


In [38]:
title.invoke('A girl chosen to be the saintess for godness of beauty escaping from the destiny of being a sacrifice for the sin of lust')

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


{'summary': StringPromptValue(text='<s><|user|>\n    Create a title for a story about A girl chosen to be the saintess for godness of beauty escaping from the destiny of being a sacrifice for the sin of lust. Only return the title.<|end|>\n    <|assistant|>'),
 'title': ' "Belle\'s Ascent: Defying Beauty\'s Sacrifice"'}

## 记忆

<https://www.langchain.com.cn/docs/how_to/message_history/>

### 对话缓冲区

直接将完整对话记录注入输入提示

控制窗口上下文：仅保留最近 k 轮的对话记录

### 对话摘要

通过另一个 LLM 模型接受完整的对话记录，提炼摘要，保留对话历史的核心

使用历史摘要注入输入提示

In [40]:
# RunnableWithMessageHistory
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory


def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")


chain = RunnableParallel({"output_message": deepseek_model})


runnable_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    output_messages_key="output_message",
)

runnable_with_history.invoke(
    [HumanMessage(content="hi - im bob!")],
    config={"configurable": {"session_id": "1"}},
)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/langchain_core/runnables/history.py:596: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  message_history = self.get_session_history(


{'output_message': AIMessage(content='Hi Bob! 👋 Nice to meet you! How can I help you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 8, 'total_tokens': 27, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 8}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': 'ca284ee6-c72f-4763-8316-d66d56c8e588', 'finish_reason': 'stop', 'logprobs': None}, id='run--32a757ad-6216-4833-8436-d7c03a8c5bff-0', usage_metadata={'input_tokens': 8, 'output_tokens': 19, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})}

## Agent：智能体

agent: 可以自主规划行动及其序列的系统

ReAct（reasoning and acting，推理与行动）框架

### 工具

<https://www.langchain.com.cn/docs/how_to/custom_tools/>



In [41]:
from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [42]:
from langchain_core.tools import StructuredTool


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator = StructuredTool.from_function(func=multiply, coroutine=amultiply)


In [43]:
# runnable as_tool
title_tool = title.as_tool()

/var/folders/9x/vrm2qywn327gf_tw665dc9w40000gn/T/ipykernel_5729/2858425346.py:2: LangChainBetaWarning: This API is in beta and may change in the future.
  title_tool = title.as_tool()


In [44]:
# impl BaseTool
from typing import Optional, Type
from pydantic import BaseModel, Field

from langchain_core.callbacks import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain_core.tools import BaseTool


class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


# Note: It's important that every field has type hints. BaseTool is a
# Pydantic class and not having type hints can lead to unexpected behavior.
class CustomCalculatorTool(BaseTool):
    name: str = "Calculator"
    description: str = "useful for when you need to answer questions about math"
    args_schema: Type[BaseModel] = CalculatorInput
    return_direct: bool = True

    def _run(
        self, a: int, b: int, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return a * b

    async def _arun(
        self,
        a: int,
        b: int,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        # If the calculation is cheap, you can just delegate to the sync implementation
        # as shown below.
        # If the sync calculation is expensive, you should delete the entire _arun method.
        # LangChain will automatically provide a better implementation that will
        # kick off the task in a thread to make sure it doesn't block other async code.
        return self._run(a, b, run_manager=run_manager.get_sync())

In [60]:
# ReAct
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchTool
from langchain_community.agent_toolkits.load_tools import load_tools
from langgraph.prebuilt.chat_agent_executor import create_react_agent
from langchain.agents.agent import AgentExecutor
# LLMMath已经废弃
tools = load_tools([], llm=deepseek_model)
tools.append(DuckDuckGoSearchTool())
tools.append(calculator)

react_template = """

Question: {messages}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["messages"]
)


agent = create_react_agent(deepseek_model, tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "What is the current price of a MacBook Pro in CNY? How much would it cost in EUR with current exchange rate?"})



> Entering new AgentExecutor chain...


Error in StdOutCallbackHandler.on_agent_action callback: AttributeError("'str' object has no attribute 'log'")


AttributeError: 'str' object has no attribute 'tool'